## Lesson 6: Semantic Search, Building a Q&A System

#### Project environment setup

- Load credentials and relevant Python Libraries

In [1]:
from utils import authenticate
credentials, PROJECT_ID = authenticate()

2025-01-10 10:30:32.987019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-10 10:30:33.067016: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-10 10:30:33.069919: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-01-10 10:30:33.069934: I tensorflow/compiler/xla

#### Enter project details

In [2]:
REGION = 'us-central1'

In [3]:
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION, credentials = credentials)

## Load Stack Overflow questions and answers from BigQuery

In [4]:
import pandas as pd

In [5]:
so_database = pd.read_csv('so_database_app.csv')

In [6]:
print("Shape: " + str(so_database.shape))
print(so_database)

Shape: (2000, 3)
                                             input_text  \
0     python's inspect.getfile returns "<string>"<p>...   
1     Passing parameter to function while multithrea...   
2     How do we test a specific method written in a ...   
3     how can i remove the black bg color of an imag...   
4     How to extract each sheet within an Excel file...   
...                                                 ...   
1995  Is it possible to made inline-block elements l...   
1996  Flip Clock code works on Codepen and doesn't w...   
1997  React Native How can I put one view in front o...   
1998  setting fixed width with 100% height of the pa...   
1999  How to make sidebar button not bring viewpoint...   

                                            output_text category  
0     <p><code>&lt;string&gt;</code> means that the ...   python  
1     <p>Try this and note the difference:</p>\n<pre...   python  
2     <p>Duplicate of <a href="https://stackoverflow...   python  
3     

## Load the question embeddings

In [7]:
from vertexai.language_models import TextEmbeddingModel

In [8]:
embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

In [9]:
import numpy as np
from utils import encode_text_to_embedding_batched

- Here is the code that embeds the text.  You can adapt it for use in your own projects.  
- To save on API calls, we've embedded the text already, so you can load it from the saved file in the next cell.

```Python
# Encode the stack overflow data

so_questions = so_database.input_text.tolist()
question_embeddings = encode_text_to_embedding_batched(
            sentences = so_questions,
            api_calls_per_second = 20/60, 
            batch_size = 5)
```

In [10]:
import pickle
with open('question_embeddings_app.pkl', 'rb') as file:
      
    # Call load method to deserialze
    question_embeddings = pickle.load(file)
  
    print(question_embeddings)

[[-0.03571156 -0.00240684  0.05860338 ... -0.03100227 -0.00855574
  -0.01997405]
 [-0.02024316 -0.0026255   0.01940405 ... -0.02158143 -0.05655403
  -0.01040497]
 [-0.05175979 -0.03712264  0.02699278 ... -0.07055898 -0.0402537
   0.00092099]
 ...
 [-0.00580394 -0.01621097  0.05829635 ... -0.03350992 -0.05343556
  -0.06016821]
 [-0.00436622 -0.02692963  0.03363771 ... -0.01686567 -0.03812337
  -0.02329491]
 [-0.04240424 -0.01633749  0.05516777 ... -0.02697376 -0.01751165
  -0.04558187]]


In [11]:
so_database['embeddings'] = question_embeddings.tolist()

In [12]:
so_database

,input_text,output_text,category,embeddings
0,"python's inspect.getfile returns ""<string>""<p>...",<p><code>&lt;string&gt;</code> means that the ...,python,"[-0.03571155667304993, -0.0024068362545222044,..."
1,Passing parameter to function while multithrea...,<p>Try this and note the difference:</p>\n<pre...,python,"[-0.020243162289261818, -0.002625499852001667,..."
2,How do we test a specific method written in a ...,"<p>Duplicate of <a href=""https://stackoverflow...",python,"[-0.05175979062914848, -0.03712264448404312, 0..."
3,how can i remove the black bg color of an imag...,<p>The alpha channel &quot;disappears&quot; be...,python,"[0.02206624671816826, -0.028208276256918907, 0..."
4,How to extract each sheet within an Excel file...,<p>You need to specify the <code>index</code> ...,python,"[-0.05498068407177925, -0.0032414537854492664,..."
...,...,...,...,...
1995,Is it possible to made inline-block elements l...,<p>If this is only for the visual purpose then...,css,"[-0.009190441109240055, -0.01732615754008293, ..."
1996,Flip Clock code works on Codepen and doesn't w...,<p>You forgot to attach the CSS file for the f...,css,"[-0.009033069014549255, -0.0009270847076550126..."
1997,React Native How can I put one view in front o...,<p>You can do it using zIndex for example:</p>...,css,"[-0.005803938489407301, -0.016210969537496567,..."
1998,setting fixed width with 100% height of the pa...,<p>You can use <code>width: calc(100% - 100px)...,css,"[-0.004366223234683275, -0.02692963369190693, ..."


## Semantic Search

When a user asks a question, we can embed their query on the fly and search over all of the Stack Overflow question embeddings to find the most simliar datapoint.

In [13]:
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances_argmin as distances_argmin

In [14]:
query = ['How to concat dataframes pandas']

In [27]:
embedding_model.get_embeddings(query)[0].values

'message'

In [19]:
[query_embedding]

['message']

In [17]:
so_database.embeddings.values

array([list([-0.03571155667304993, -0.0024068362545222044, 0.05860338360071182, 0.021544553339481354, 0.03684587404131889, -0.024007413536310196, 0.029984410852193832, -0.0070322779938578606, 0.03852386400103569, 0.02319701388478279, -0.014075829647481441, 0.019536204636096954, 0.02271679788827896, 0.011844916269183159, 0.006728676147758961, -0.015157368034124374, -0.061121731996536255, -0.024660667404532433, -0.02678326517343521, 0.00457220571115613, -0.07970225065946579, -0.007075136061757803, 0.021359460428357124, 0.032114941626787186, -0.02134050615131855, -0.0856335461139679, 0.0004826547228731215, -0.01045012567192316, -0.024015050381422043, 0.0031585979741066694, -0.008200902491807938, 0.04345826432108879, -0.032868675887584686, -0.026998793706297874, 0.005545898340642452, -0.0014554146910086274, 0.003762691281735897, -0.0024402241688221693, 0.004386576358228922, 0.015530101954936981, 0.018350347876548767, -0.0016730629140511155, 0.0808154046535492, 0.04275016859173775, -0.00214

In [18]:
cos_sim_array = cosine_similarity([query_embedding],
                                  list(so_database.embeddings.values))

ValueError: could not convert string to float: 'message'

In [ ]:
cos_sim_array.shape

Once we have a similarity value between our query embedding and each of the database embeddings, we can extract the index with the highest value. This embedding corresponds to the Stack Overflow post that is most similiar to the question "How to concat dataframes pandas".

In [ ]:
index_doc_cosine = np.argmax(cos_sim_array)

In [ ]:
index_doc_distances = distances_argmin([query_embedding], 
                                       list(so_database.embeddings.values))[0]

In [ ]:
so_database.input_text[index_doc_cosine]

In [ ]:
so_database.output_text[index_doc_cosine]

## Question answering with relevant context

Now that we have found the most simliar Stack Overflow question, we can take the corresponding answer and use an LLM to produce a more conversational response.

In [28]:
from vertexai.language_models import TextGenerationModel

In [29]:
generation_model = TextGenerationModel.from_pretrained(
    "text-bison@001")

In [30]:
context = "Question: " + so_database.input_text[index_doc_cosine] +\
"\n Answer: " + so_database.output_text[index_doc_cosine]

NameError: name 'index_doc_cosine' is not defined

In [ ]:
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """

In [ ]:
from IPython.display import Markdown, display

t_value = 0.2
response = generation_model.predict(prompt = prompt,
                                    temperature = t_value,
                                    max_output_tokens = 1024)

display(Markdown(response.text))

## When the documents don't provide useful information

Our current workflow returns the most similar question from our embeddings database. But what do we do when that question isn't actually relevant when answering the user query? In other words, we don't have a good match in our database.

In addition to providing a more conversational response, LLMs can help us handle these cases where the most similiar document isn't actually a reasonable answer to the user's query.

In [ ]:
query = ['How to make the perfect lasagna']

In [ ]:
query_embedding = embedding_model.get_embeddings(query)[0].values

In [ ]:
cos_sim_array = cosine_similarity([query_embedding], 
                                  list(so_database.embeddings.values))

In [ ]:
cos_sim_array

In [ ]:
index_doc = np.argmax(cos_sim_array)

In [ ]:
context = so_database.input_text[index_doc] + \
"\n Answer: " + so_database.output_text[index_doc]

In [ ]:
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, answer with 
             [I couldn't find a good match in the \
             document database for your query]
             """

In [ ]:
t_value = 0.2
response = generation_model.predict(prompt = prompt,
                                    temperature = t_value,
                                    max_output_tokens = 1024)
display(Markdown(response.text))

## Scale with approximate nearest neighbor search

When dealing with a large dataset, computing the similarity between the query and each original embedded document in the database might be too expensive. Instead of doing that, you can use approximate nearest neighbor algorithms that find the most similar documents in a more efficient way.

These algorithms usually work by creating an index for your data, and using that index to find the most similar documents for your queries. In this notebook, we will use ScaNN to demonstrate the benefits of efficient vector similarity search. First, you have to create an index for your embedded dataset.

In [ ]:
import scann
from utils import create_index

#Create index using scann
index = create_index(embedded_dataset = question_embeddings, 
                     num_leaves = 25,
                     num_leaves_to_search = 10,
                     training_sample_size = 2000)

In [ ]:
query = "how to concat dataframes pandas"

In [ ]:
import time 

start = time.time()
query_embedding = embedding_model.get_embeddings([query])[0].values
neighbors, distances = index.search(query_embedding, final_num_neighbors = 1)
end = time.time()

for id, dist in zip(neighbors, distances):
    print(f"[docid:{id}] [{dist}] -- {so_database.input_text[int(id)][:125]}...")

print("Latency (ms):", 1000 * (end - start))

In [ ]:
start = time.time()
query_embedding = embedding_model.get_embeddings([query])[0].values
cos_sim_array = cosine_similarity([query_embedding], list(so_database.embeddings.values))
index_doc = np.argmax(cos_sim_array)
end = time.time()

print(f"[docid:{index_doc}] [{np.max(cos_sim_array)}] -- {so_database.input_text[int(index_doc)][:125]}...")

print("Latency (ms):", 1000 * (end - start))